In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt

# Mediapipe Face Mesh 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 얼굴 랜드마크 좌표를 추출하는 함수
def extract_face_landmarks(image):
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.1, min_tracking_confidence=0.1) as face_mesh:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)

        landmarks = []
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for landmark in face_landmarks.landmark:
                    landmarks.append((landmark.x, landmark.y, landmark.z))
        return landmarks

# 얼굴 랜드마크 좌표를 OBJ 형식으로 저장하는 함수
def save_landmarks_to_obj(landmarks, obj_filename):
    # 얼굴 랜드마크 좌표를 NumPy 배열로 변환
    vertices = np.array(landmarks)

    # Delaunay 삼각화
    tri = Delaunay(vertices[:, :2])

    # OBJ 파일 작성
    with open(obj_filename, 'w') as obj_file:
        for vertex in landmarks:
            obj_file.write(f'v {vertex[0]} {vertex[1]} {vertex[2]}\n')
        for simplex in tri.simplices:
            obj_file.write(f'f {simplex[0]+1} {simplex[1]+1} {simplex[2]+1}\n')

# 이미지 로드
image_path = 'test_img.jpg'
image = cv2.imread(image_path)

# 얼굴 랜드마크 추출
face_landmarks = extract_face_landmarks(image)

# 얼굴 랜드마크를 OBJ 파일로 저장
if face_landmarks:
    obj_filename = 'output_mesh.obj'
    save_landmarks_to_obj(face_landmarks, obj_filename)
    print(f'Successfully saved the 3D mesh to {obj_filename}')
else:
    print('No face landmarks detected.')
